In [67]:
# from google.colab import drive
# drive.mount('/content/drive')

In [68]:
# pip install seaborn

# !pip install tensorflow


In [69]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [70]:
#To read traing data of google share price
df=pd.read_csv('/content/drive/MyDrive/BTC_Prediction_Training/data/btc.csv')
#training_set user defined dataframe
df.head()

,Unnamed: 0,Date,Close,High,Low,Open,Volume
0,0,2022-01-01,47686.812500,47827.312500,46288.484375,46311.746094,24582667004
1,1,2022-01-02,47345.218750,47881.406250,46856.937500,47680.925781,27951569547
2,2,2022-01-03,46458.117188,47510.726562,45835.964844,47343.542969,33071628362
3,3,2022-01-04,45897.574219,47406.546875,45752.464844,46458.851562,42494677905
4,4,2022-01-05,43569.003906,46929.046875,42798.222656,45899.359375,36851084859


In [71]:
# drop column 'Unnamed: 0'
df.drop('Unnamed: 0',axis = 1, inplace= True)

In [72]:
df['Date'].min()

'2022-01-01'

In [73]:
df['Date'].max()

'2025-02-28'

In [74]:
df.dtypes

,0
Date,object
Close,float64
High,float64
Low,float64
Open,float64
Volume,int64


In [75]:
df['Date'] = pd.to_datetime(df['Date'])
df.head()



,Date,Close,High,Low,Open,Volume
0,2022-01-01,47686.812500,47827.312500,46288.484375,46311.746094,24582667004
1,2022-01-02,47345.218750,47881.406250,46856.937500,47680.925781,27951569547
2,2022-01-03,46458.117188,47510.726562,45835.964844,47343.542969,33071628362
3,2022-01-04,45897.574219,47406.546875,45752.464844,46458.851562,42494677905
4,2022-01-05,43569.003906,46929.046875,42798.222656,45899.359375,36851084859


In [76]:
df.dtypes

,0
Date,datetime64[ns]
Close,float64
High,float64
Low,float64
Open,float64
Volume,int64


In [77]:
df.isnull().sum()

,0
Date,0
Close,0
High,0
Low,0
Open,0
Volume,0


In [78]:
#separate Open column from given dataset
df1=df.iloc[:,1:2].values #or df1=df['Open']  df.loc[:,'Open']
#1:2 means start=1 and stop=2-1=1
type(df1)


numpy.ndarray

In [79]:
df1

array([[47686.8125   ],
       [47345.21875  ],
       [46458.1171875],
       ...,
       [84347.0234375],
       [84704.2265625],
       [84373.0078125]])

In [80]:
df1.ndim

2

In [81]:
# Feature Scaling is most important part of data pre-processing
# Stock market data : low-high
# Mostly apply Normalisation: means min-max-scaler

from sklearn.preprocessing import MinMaxScaler

#create object of MinMaxScaler class
mms=MinMaxScaler()
df1=mms.fit_transform(df1)

In [82]:
df1

array([[0.35303107],
       [0.34925067],
       [0.33943314],
       ...,
       [0.75874847],
       [0.76270163],
       [0.75903604]])

In [83]:
#Find the total number of records
df1.shape

(1155, 1)

In [84]:
#separate Input and output from Open column
X_train=df1[0:1154] #start=0 steop=1154-1=1155
Y_train=df1[1:1155]

In [85]:
Y_train.shape

(1154, 1)

###### Now that we have our training data, the next step is to reshape it into the 3D format required by LSTM models.
 This format includes the number of samples, the number of timesteps, and the number of features.

In [86]:
X_train.ndim

2

In [87]:
#no. of rows ,no. of columns,time stamp
#1257,1,1
#next step : we have going to reshape of our data
#3 dimensional data should be required .
#first dimension show no. of rows means here 1257 rows
#second dimension : show no. of columns means no. of features(input) :here 1
#third dimension : corresponds to a timestamp
#because our input is t (current) and output is t+1(next day). so that
#time stamp =t+1-t=1 .its represent time stamp
#suppose stock price after 4 days means we predict output t+4 means
#time stamp=t+4-t=4

#We provide 3D data then we converts our input X_train data into 3D
X_train = np.reshape(X_train, (1154, 1, 1))


In [88]:
#To check dimension
X_train.ndim

3

In [89]:
X_train

array([[[0.35303107]],

       [[0.34925067]],

       [[0.33943314]],

       ...,

       [[0.80732304]],

       [[0.75874847]],

       [[0.76270163]]])

In [90]:
#Part-2 : -
#Building the RNN with LSTM
#Recurrent neural network
#LSTM means long short term memory
from tensorflow.keras.models import Sequential #initialise neural network
from tensorflow.keras.layers import Dense #to add input layer,output layer,
#and hidden layer
from tensorflow.keras.layers import LSTM  #to use built the model
#LSTM : to manage internal memory , to store previous o/p
#long short term memory


In [91]:
#We have regression problem means our output is contineous numeric value
#means to predict contineous numeric value  means this is regression problem

In [94]:
#create objecgt of Sequential class
model=Sequential()

#To add layers in model
#This layer are input layer and LSTM layer
model.add(LSTM(units=4,activation='sigmoid',input_shape=(None,1)))
#None means to take no. of rows and 1 means 1 column

#To add output layer
model.add(Dense(units=1,activation='linear'))

#compile the model
model.compile(optimizer='adam',loss='mean_squared_error')


model.summary()

#early stopping
from tensorflow.keras.callbacks import EarlyStopping
#procedure for callback(EarlyStopping)
callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.00001,
    patience=20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False
)


#Train the model
trained_model=model.fit(X_train, Y_train, batch_size=10, epochs=75,
                            validation_split=0.2, callbacks=[callback])
#validation_split=0.2 means 20% data of training data X_train
#80% for training and 20% for validation

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 4)              │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101 (404.00 B)

 Trainable params: 101 (404.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/75
93/93 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0259 - val_loss: 0.2364
Epoch 2/75
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0224 - val_loss: 0.2206
Epoch 3/75
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0225 - val_loss: 0.2126
Epoch 4/75
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0207 - val_loss: 0.2034
Epoch 5/75
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0212 - val_loss: 0.1876
Epoch 6/75
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0186 - val_loss: 0.1684
Epoch 7/75
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0169 - val_loss: 0.1588
Epoch 8/75
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0156 - val_loss: 0.1513
Epoch 9/75
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0140 - val_loss: 0.1335
Epoch 10/75
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0132 - val_loss: 0.1177
Epoch 11/75
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0107 - val_loss: 0.1052
Epoch 12/75
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0101 - val_lo

In [95]:
# If we want to predict stock price of next 5 day's , then we use for loop

n=float(input('Enter Today Opening Stock Price :'))
# here n is scaler variable
# print(type(n))
# First, convert n into 2D numpy array
n=np.array(n).reshape(-1,1)
days=int(input('Enter How many days'))
for i in range(days):
  n=np.array(n).reshape(-1,1)   # here -1 means None means automatic accept no. or rows but next 1 means no. of column
  # Apply scaling : normalisation means MinMaxScaler
  n1=mms.transform(n)
  # converts n1 from 2D into 3D array
  v=n1.reshape(-1,1,1)
  # prediction
  pred=model.predict(v)
  # to remove scaling
  final=mms.inverse_transform(pred)
  print('Opening Price of Next day: ',final)
  n=final

Enter Today Opening Stock Price :46311.746094
Enter How many days3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
Opening Price of Next day:  [[46749.89]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Opening Price of Next day:  [[47189.336]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Opening Price of Next day:  [[47629.504]]
